In [ ]:
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
from math import sin, cos, pi, sqrt, atan2
import random
import time
import copy
import os
import sys
sys.path.insert(1, '../framework')
from racetrack import *
rt = RACETrack()
pts = [(20,20),(100,20),(100,100), (110,100), (300,100), (310,210), (310,300), (320,300), (330,300), (340,290), (350,280), (350, 50), (360,40), (370,30), (380,40), (390,50), (400,400)]

In [ ]:
svg = ['<svg x="0" y="0" width="500" height="500">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#000000" />')
pw = rt.piecewiseCubicBSpline(pts)
_colors_       = rt.co_mgr.brewerColors('qualitative', n=10)
_stroke_width_ = 2.0
_gap_          = 2.0
for i in range(len(pw)-1):
    x0, y0, x1, y1 = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
    dx, dy = x1 - x0, y1 - y0
    _len_  = sqrt(dx**2 + dy**2)
    if _len_ < 0.001: _len_ = 0.001
    dx,  dy  = dx/_len_, dy/_len_
    pdx, pdy = dy, -dx
    for j in range(10):
        _x0_, _y0_, _x1_, _y1_ = x0, y0, x1, y1
        if j > 0:
            if j%2 == 0:
                _x0_ += pdx * (_gap_ + _stroke_width_) * (j/2)
                _y0_ += pdy * (_gap_ + _stroke_width_) * (j/2)
                _x1_ += pdx * (_gap_ + _stroke_width_) * (j/2)
                _y1_ += pdy * (_gap_ + _stroke_width_) * (j/2)
            else:
                _x0_ -= pdx * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y0_ -= pdy * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _x1_ -= pdx * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y1_ -= pdy * (_gap_ + _stroke_width_) * (int(j/2)+1)
        _co_ = _colors_[j]
        svg.append(f'<line x1="{_x0_}" y1="{_y0_}" x2="{_x1_}" y2="{_y1_}" stroke="{_co_}" stroke-width="{_stroke_width_}" />')
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
#
# This came from screwing up the math... but it looks good :( :)
# -- it's because i re-used pdx, pdy .... which were set above... and are fixed...
#
svg = ['<svg x="0" y="0" width="500" height="500">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#000000" />')
pw = rt.piecewiseCubicBSpline(pts)
_colors_       = rt.co_mgr.brewerColors('qualitative', n=10)
_stroke_width_ = 4.0
_gap_          = 1.0
for j in range(10):
    _co_                 = _colors_[j]
    _x1_last_, _y1_last_ = None, None
    _path_               = []
    for i in range(len(pw)-1):
        x0_base, y0_base, x1_base, y1_base = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
        dx_base, dy_base = x1_base - x0_base, y1_base - y0_base
        _len_  = sqrt(dx_base**2 + dy_base**2)
        if _len_ < 0.001: continue # _len_ = 0.001 # appears to be an error in the piecewiseCubicBSpline where zero length segments are created
        dx_base,  dy_base  = dx_base/_len_,  dy_base/_len_
        pdx_base, pdy_base = dy_base,       -dx_base
        _x0_, _y0_, _x1_, _y1_ = x0_base, y0_base, x1_base, y1_base
        if j > 0:
            if j%2 == 0:
                _x0_ += pdx * (_gap_ + _stroke_width_) * (j/2)
                _y0_ += pdy * (_gap_ + _stroke_width_) * (j/2)
                _x1_ += pdx * (_gap_ + _stroke_width_) * (j/2)
                _y1_ += pdy * (_gap_ + _stroke_width_) * (j/2)
            else:
                _x0_ -= pdx * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y0_ -= pdy * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _x1_ -= pdx * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y1_ -= pdy * (_gap_ + _stroke_width_) * (int(j/2)+1)
        _dx_local_, _dy_local_ = _x1_ - _x0_, _y1_ - _y0_
        _len_  = sqrt(_dx_local_**2 + _dy_local_**2)
        if _len_ < 0.001: continue
        _dx_local_, _dy_local_ = _dx_local_/_len_, _dy_local_/_len_    
        if _x1_last_ is not None: 
            _path_.append(f'L {_x0_} {_y0_} ')
            _path_.append(f'L {_x1_} {_y1_} ')
        else: _path_.append(f'M {_x0_} {_y0_} ')
        _x1_last_, _y1_last_ = _x1_, _y1_
    _path_description_ = ''.join(_path_)
    svg.append(f'<path d="{_path_description_}" stroke="{_co_}" fill="none" stroke-width="{_stroke_width_}" />')
svg.append('</svg>')
rt.tile([''.join(svg)])

In [ ]:
svg = ['<svg x="0" y="0" width="500" height="500">']
svg.append('<rect x="0" y="0" width="500" height="500" fill="#000000" />')
pw = rt.piecewiseCubicBSpline(pts)
_colors_       = rt.co_mgr.brewerColors('qualitative', n=10)
_stroke_width_ =  4.0
_gap_          = 20.0

_lu_ = {'pt':[], 'path':[], 'dot':[]}
for i in range(len(_colors_)): 
    rt.co_mgr.str_to_color_lu[i]      = _colors_[i]
    rt.co_mgr.str_to_color_lu[str(i)] = _colors_[i]

for j in range(3):
    _co_                           = _colors_[j]
    _x1_base_last_, _y1_base_last_ = None, None
    _x1_last_,      _y1_last_      = None, None
    _path_               = []
    for i in range(len(pw)-1):
        x0_base, y0_base, x1_base, y1_base = pw[i][0], pw[i][1], pw[i+1][0], pw[i+1][1]
        dx_base, dy_base = x1_base - x0_base, y1_base - y0_base
        _len_  = sqrt(dx_base**2 + dy_base**2)
        if _len_ < 0.001: continue # _len_ = 0.001 # appears to be an error in the piecewiseCubicBSpline where zero length segments are created
        dx_base,  dy_base  = dx_base/_len_,  dy_base/_len_
        pdx_base, pdy_base = dy_base,       -dx_base
        _x0_, _y0_, _x1_, _y1_ = x0_base, y0_base, x1_base, y1_base
        if j > 0:
            if j%2 == 0:
                _x0_ += pdx_base * (_gap_ + _stroke_width_) * (j/2)
                _y0_ += pdy_base * (_gap_ + _stroke_width_) * (j/2)
                _x1_ += pdx_base * (_gap_ + _stroke_width_) * (j/2)
                _y1_ += pdy_base * (_gap_ + _stroke_width_) * (j/2)
            else:
                _x0_ -= pdx_base * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y0_ -= pdy_base * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _x1_ -= pdx_base * (_gap_ + _stroke_width_) * (int(j/2)+1)
                _y1_ -= pdy_base * (_gap_ + _stroke_width_) * (int(j/2)+1)
        _dx_local_, _dy_local_ = _x1_ - _x0_, _y1_ - _y0_
        _len_  = sqrt(_dx_local_**2 + _dy_local_**2)
        if _len_ < 0.001: continue
        _dx_local_, _dy_local_ = _dx_local_/_len_, _dy_local_/_len_

        render = True
        if _x1_base_last_ is not None:
            _my_dx_, _my_dy_ = _x0_ - _x1_last_, _y0_ - _y1_last_
            _len_ = sqrt(_my_dx_**2 + _my_dy_**2)
            if _len_ < 0.001: _len_ = 0.001
            _my_dx_, _my_dy_ = _my_dx_/_len_, _my_dy_/_len_
            #_lu_['pt'].append(i), _lu_['path'].append(j), _lu_['dot'].append(_my_dx_*_my_dy_ - _dx_local_*_dy_local_)
            _lu_['pt'].append(i), _lu_['path'].append(j), _lu_['dot'].append(atan2(_my_dy_, _my_dx_) - atan2(_dy_local_, _dx_local_))

        if render:
            if len(_path_) == 0: 
                _path_.append(f'M {_x0_} {_y0_} ')
                _path_.append(f'L {_x1_} {_y1_} ')
            else:
                _path_.append(f'L {_x0_} {_y0_} ')
                _path_.append(f'L {_x1_} {_y1_} ')

        _x1_base_last_, _y1_base_last_ = x1_base, y1_base
        _x1_last_,      _y1_last_      = _x1_,    _y1_
    _path_description_ = ''.join(_path_)
    svg.append(f'<path d="{_path_description_}" stroke="{_co_}" fill="none" stroke-width="{_stroke_width_}" stroke-linecap="butt" />')
svg.append('</svg>')
df_debug = pd.DataFrame(_lu_)
rt.tile([''.join(svg), rt.xy(df_debug, x_field='pt', y_field='dot', color_by='path', w=1024, h=500), rt.histogram(df_debug, bin_by='path', color_by='path', h=500)])